# Train model in SFT fashion

In this notebook, there are different aproaches for Supervised fine-tuning

## Read Libraries

In [ ]:
! pip install transformers datasets trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.9/133.9 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 8.3 MB/s eta 0:00:00


In [ ]:
! pip install peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 3.3 MB/s eta 0:00:00


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer,AutoModelWithLMHead,GPT2Model,TrainingArguments
from datasets import load_dataset
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
import pandas as pd
from datasets import Dataset
from peft import PeftModel, PeftConfig
import random

/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##Prepare the data

The data needed to be loaded ftom Google drive and the bineary sentiment needed to be transformed to the description (positive and negative). Also for testing two data sets have been created. One for training and another one for validation.

In [ ]:
 path="/content/drive/MyDrive/Masterthesis/Data/data_processed/train_for_sft.tsv"
 data=pd.read_csv(path, sep='\t')
 data=data[["preprocessed_text","sentiment"]]
 data

,preprocessed_text,sentiment
0,Es gibt so viele verschiedene Freizeitaktivitä...,1
1,Das Hotel kann man nicht weiter empfehlen. Das...,0
2,"Regisseur John Polson bemüht sich, Spannung au...",0
3,Absolut nicht zu empfehlen!!! Die Lage des Hau...,0
4,"Die Lage ist der Hammer. Zum Strand, der natür...",1
...,...,...
73601,Ganz einfach: Super! Ich habe selten in einem ...,1
73602,"Animation: Es gibt zur Zeit nur Animateure, di...",0
73603,"Super Hotel, werden Wiederholungstäter Die Hot...",1
73604,Katalog und Realität stimmen nicht überein Hot...,0


In [ ]:
def sentiment_map(x):
  if x==1:
    return "positive"
  if x==0:
    return "negative"

In [ ]:
data["sentiment"]=data["sentiment"].apply(lambda x:sentiment_map(x) )



In [ ]:
 path_val="/content/drive/MyDrive/Masterthesis/Data/data_processed/val_for_sft.tsv"
 data_val=pd.read_csv(path_val, sep='\t')
 data_val=data_val[["preprocessed_text","sentiment"]]


In [ ]:
data_val["sentiment"]=data_val["sentiment"].apply(lambda x:sentiment_map(x) )


### First Approach

The first model should create the possibility to generate text with given sentiment in the "[]" as a token. For this a column "instructions has been created.  Ith as two pattern :Instruction and Response. In instruction part, the humnan should be able to specity the desired sentiment and frist words to use and the Assistant should generate response. This approach was not able to generate sentiment based responses.

Not included to thesis.

In [ ]:
def create_instruction(sent,text):
  random_integer = random.randint(2, 4)
  words = text.split()
  begin_sent=' '. join(words[:random_integer])
  text=f"### Human:[{sent}]: {begin_sent}### Assistant: {text}"
  words = text.split()
  text =' '. join(words[:512])
  return text

In [ ]:
data["instructions"]=data.apply(lambda row: create_instruction(row['sentiment'], row['preprocessed_text']), axis=1)
data["instructions"]

0        ### Human:[positive]: Es gibt so### Assistant:...
1        ### Human:[negative]: Das Hotel### Assistant: ...
2        ### Human:[negative]: Regisseur John### Assist...
3        ### Human:[negative]: Absolut nicht zu empfehl...
4        ### Human:[positive]: Die Lage ist### Assistan...
                               ...                        
73601    ### Human:[positive]: Ganz einfach: Super! Ich...
73602    ### Human:[negative]: Animation: Es gibt### As...
73603    ### Human:[positive]: Super Hotel, werden Wied...
73604    ### Human:[negative]: Katalog und Realität sti...
73605    ### Human:[negative]: Unfreundlich, Qualität u...
Name: instructions, Length: 73606, dtype: object

In [ ]:
dataset = Dataset.from_pandas(data)
dataset

Dataset({
    features: ['preprocessed_text', 'sentiment', 'instructions'],
    num_rows: 73606
})

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("benjamin/gpt2-wechsel-german")

model = AutoModelWithLMHead.from_pretrained("benjamin/gpt2-wechsel-german")

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1499: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [ ]:
tokenizer.pad_token = tokenizer.eos_token


In [ ]:
instruction_template = "### Human:"
response_template = "### Assistant:"
collator = DataCollatorForCompletionOnlyLM(instruction_template=instruction_template, response_template=response_template, tokenizer=tokenizer, mlm=False)


In [ ]:
trainer = SFTTrainer(
    model,
    train_dataset=dataset,
    dataset_text_field="instructions",
    max_seq_length=512,
    data_collator=collator,
)

Using pad_token, but it is not set yet.


Map:   0%|          | 0/14534 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,3.716300
1000,3.615000
1500,3.566700
2000,3.416700
2500,3.303400
3000,3.288700
3500,3.267500
4000,3.185400
4500,3.160500
5000,3.122800


TrainOutput(global_step=5451, training_loss=3.3463986637701226, metrics={'train_runtime': 882.5809, 'train_samples_per_second': 49.403, 'train_steps_per_second': 6.176, 'total_flos': 6296097906432000.0, 'train_loss': 3.3463986637701226, 'epoch': 3.0})

In [ ]:
trainer.save_model("/content/drive/MyDrive/Masterthesis/Models/german_gpt2_sft")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/Masterthesis/Models/german_gpt2_sft")

model = AutoModelWithLMHead.from_pretrained("/content/drive/MyDrive/Masterthesis/Models/german_gpt2_sft")

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1479: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [ ]:
prompt_sent="[negative]:Das Hotel"

In [ ]:
generation_kwargs = {
    "min_length": -1,
    "top_k": 5,
    "top_p": 0,
    "do_sample": True,
    "pad_token_id": tokenizer.eos_token_id,
    "max_new_tokens": 56,
    "eos_token_id": -1,
}

In [ ]:
input_ids = tokenizer.encode(prompt_sent, return_tensors="pt")

In [ ]:
output = model.generate(   input_ids,
  **generation_kwargs)

generated_text = tokenizer.decode(output[0])
print("Generated Text:")
print(generated_text)


Generated Text:
[negative]:Das Hotel ist sehr weitläufig und man kann sich auch gut auf den Weg zum Strand machen. Der Strand ist sehr schön und sauber. Die Liegen sind sehr beqüm und man kann sich auch gut auf den Weg zum Strand machen. Die Strandbar ist sehr gut und man kann auch gut


In [ ]:
tokenizer_oryginal = AutoTokenizer.from_pretrained("benjamin/gpt2-wechsel-german")

model_oryginal = AutoModelWithLMHead.from_pretrained("benjamin/gpt2-wechsel-german")

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1479: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [ ]:
prompt_oryginal="Das Hotel "
input_ids_oryginal = tokenizer_oryginal.encode(prompt_oryginal, return_tensors="pt")

In [ ]:
output = model_oryginal.generate(   input_ids_oryginal,
  **generation_kwargs)

generated_text = tokenizer_oryginal.decode(output[0])
print("Generated Text:")
print(generated_text)

Generated Text:
Das Hotel ragt in die Höhe, aber auch in die Tiefe. Der Turm ist überdacht, die Aussicht über die Stadt und die Landschaft ist spektakulär. Das Personal ist freundlich und hilfsbereit. Die Zimmer sind sauber und zweckmäßig. Das Personal ist sehr freundlich und hilfsbereit. Die Lage ist gut,


###Second Approach - these models were selected for RL

Here the sentiment is created while training the model on the new instruction column, that has been created as the combination of the sentiment token and the sentence. This approach generates sentiment based sentences.


Selected for thesis.

In [ ]:
def create_instruction(sent,text):
  '''
  create instructions that will be used for supervised fien-tuning.
  As an instruction we create text in the form "[sentiment] text".
  Finally we take first 512 words as the instruction.
  '''

  text=f"[{sent}] {text}"
  words = text.split()
  text =' '. join(words[:512])
  return text

In [ ]:
data["instructions"]=data.apply(lambda row: create_instruction(row['sentiment'], row['preprocessed_text']), axis=1)
data["instructions"]

0        [positive] Es gibt so viele verschiedene Freiz...
1        [negative] Das Hotel kann man nicht weiter emp...
2        [negative] Regisseur John Polson bemüht sich, ...
3        [negative] Absolut nicht zu empfehlen!!! Die L...
4        [positive] Die Lage ist der Hammer. Zum Strand...
                               ...                        
73601    [positive] Ganz einfach: Super! Ich habe selte...
73602    [negative] Animation: Es gibt zur Zeit nur Ani...
73603    [positive] Super Hotel, werden Wiederholungstä...
73604    [negative] Katalog und Realität stimmen nicht ...
73605    [negative] Unfreundlich, Qualität und Angebot ...
Name: instructions, Length: 73606, dtype: object

In [ ]:
dataset = Dataset.from_pandas(data)#create dataset from pandas dataset
dataset

Dataset({
    features: ['preprocessed_text', 'sentiment', 'instructions'],
    num_rows: 73606
})

create tokenizer and model:

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("benjamin/gpt2-wechsel-german")
model = AutoModelWithLMHead.from_pretrained("benjamin/gpt2-wechsel-german")

In [ ]:
tokenizer.pad_token = tokenizer.eos_token


Add training configs. Here we train the model for 1,2 and 3 epochs.

In [ ]:
config=TrainingArguments(
    output_dir="test",
    learning_rate=5e-5,
    num_train_epochs=3###change to 1,2,3

)

In [ ]:
trainer = SFTTrainer(
    model,
    train_dataset=dataset,
    dataset_text_field="instructions",
    max_seq_length=1024,
    args=config
)

Map:   0%|          | 0/73606 [00:00<?, ? examples/s]

In [ ]:
trainer.train()


You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,3.174100
1000,3.171300
1500,3.193600
2000,3.202200
2500,3.203600
3000,3.231500
3500,3.256000
4000,3.252300
4500,3.260600
5000,3.285300


TrainOutput(global_step=27603, training_loss=3.1648802103410834, metrics={'train_runtime': 4252.0965, 'train_samples_per_second': 51.932, 'train_steps_per_second': 6.492, 'total_flos': 3.0273064155648e+16, 'train_loss': 3.1648802103410834, 'epoch': 3.0})

In [ ]:
trainer.save_model("/content/drive/MyDrive/Masterthesis/Models/german_gpt2_sft_with_tokens_big_3_epochs")

Here the model is tested. As an example we take a sentiment token and the beginning of sentence.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/Masterthesis/Models/german_gpt2_sft_with_tokens_big_3_epochs")
model = AutoModelWithLMHead.from_pretrained("/content/drive/MyDrive/Masterthesis/Models/german_gpt2_sft_with_tokens_big_3_epochs")

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1509: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [ ]:
tokenizer.pad_token = tokenizer.eos_token


In [ ]:
prompt_sent="[negative]: Obwohl ich mir Mühe gegeben habe,"

In [ ]:
generation_kwargs = {
    "min_length": -1,
    "top_k": 3,
    "top_p": 0.5,
    "do_sample": True,
    "pad_token_id": tokenizer.eos_token_id,
    "max_new_tokens": 100,
    "eos_token_id": -1,
}

In [ ]:
generation_kwargs = {
    "do_sample": True,
    "pad_token_id": tokenizer.eos_token_id,
    "max_new_tokens": 56,
    "num_beams":4,
    "no_repeat_ngram_size":3,
    "num_return_sequences":1
}

In [ ]:
input_ids = tokenizer.encode(prompt_sent, return_tensors="pt")

In [ ]:
output = model.generate(   input_ids,
  **generation_kwargs)

generated_text = tokenizer.decode(output[0])
print("Generated Text:")
print(generated_text)

Generated Text:
[negative]: Obwohl ich mir Mühe gegeben habe, war das Essen nicht gut. Es gab jeden Tag das gleiche. Die Auswahl war sehr gering, es gab nur eine Sorte Fleisch und eine Sorte Gemüse. Zudem gab es jeden Tag Nudeln mit Tomatensoße. Das Frühstück war ok, aber das Abendessen war auch nicht besser. Jeden Tag


### Third model
Model fine-tuned without sentiment token.

Not selected for thesis.

In this section, the model is slightly fine-tuned, so it knows the data a bit, but do not overfit. To avvoid overfitting the model was fine-tuned for only one epoch.

In [ ]:
dataset = Dataset.from_pandas(data)


In [ ]:
dataset_val = Dataset.from_pandas(data_val)


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("benjamin/gpt2-wechsel-german")

model = AutoModelWithLMHead.from_pretrained("benjamin/gpt2-wechsel-german")

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1479: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [ ]:
tokenizer.pad_token = tokenizer.eos_token


In [ ]:
config=TrainingArguments(
    output_dir="test",
    learning_rate=5e-5,
    num_train_epochs=1

)

In [ ]:
trainer = SFTTrainer(
    model,
    train_dataset=dataset,
   # eval_dataset=dataset_val,
    dataset_text_field="preprocessed_text",
    max_seq_length=1024,
    args=config
)

Using pad_token, but it is not set yet.


Map:   0%|          | 0/73606 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,3.748800
1000,3.787800
1500,3.740800
2000,3.687400
2500,3.637800
3000,3.627200
3500,3.619600
4000,3.586000
4500,3.570400
5000,3.569600


TrainOutput(global_step=9201, training_loss=3.596891018660506, metrics={'train_runtime': 1429.7763, 'train_samples_per_second': 51.481, 'train_steps_per_second': 6.435, 'total_flos': 1.0202794495488e+16, 'train_loss': 3.596891018660506, 'epoch': 1.0})

In [ ]:
trainer.save_model("/content/drive/MyDrive/Masterthesis/Models/german_gpt2_sft_simple_1_epochs")

In [ ]:
tokenizer_oryginal = AutoTokenizer.from_pretrained("/content/drive/MyDrive/Masterthesis/Models/german_gpt2_sft_simple_1_epochs")

model_oryginal = AutoModelWithLMHead.from_pretrained("/content/drive/MyDrive/Masterthesis/Models/german_gpt2_sft_simple_1_epochs")

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1479: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [ ]:
prompt_oryginal="Der neue Koch"
input_ids_oryginal = tokenizer_oryginal.encode(prompt_oryginal, return_tensors="pt")

In [ ]:
generation_kwargs = {
    "top_k": 4,
    "top_p": 0.5,
    "do_sample": True,
    "pad_token_id": tokenizer_oryginal.eos_token_id,
    "max_new_tokens": 56,
    "num_beams":4,
    "no_repeat_ngram_size":3,
    "num_return_sequences":1
}

In [ ]:
output = model_oryginal.generate(input_ids_oryginal, **generation_kwargs)

generated_text = tokenizer_oryginal.decode(output[0])
print("Generated Text:")
print(generated_text)

Generated Text:
Der neue Koch ist ein absoluter Traum. Er ist sehr freundlich und zuvorkommend. Er hat immer ein offenes Ohr für alle Fragen und Wünsche. Er macht seine Arbeit sehr gut und ist immer für einen Plausch zu haben. Er spricht perfekt deutsch und ist sehr kinderlieb. Er war immer sehr freundlich,
